In [ ]:
# import sys  
# reload(sys)  
# sys.setdefaultencoding('utf8')  

In [ ]:
import numpy as np
import pandas as pd
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer('english')

In [ ]:
df_train = pd.read_csv('train.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('product_descriptions.csv', encoding="ISO-8859-1")
df_attr = pd.read_csv('attributes.csv')

### 1.Exploration

In [ ]:
num_train = df_train.shape[0]
num_train

In [ ]:
num_test = df_test.shape[0]
num_test

In [ ]:
num_pro_desc = df_pro_desc.shape[0]
num_pro_desc

In [ ]:
num_attr = df_attr.shape[0]
num_attr

In [ ]:
def str_stemmer(s):
    return ' '.join([stemmer.stem(word) for word in s.lower().split()])

In [ ]:
# if there are str1 in str2
def str_common_word(str1,str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())   # int(True)=1   

In [ ]:
df_all = pd.concat([df_train, df_test], axis = 0, ignore_index = True)

In [ ]:
#merge product description
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

In [ ]:
df_all.head()

In [ ]:
# clean: remove affixes
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))

In [ ]:
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)

In [ ]:
df_all['commons_in_title'] = df_all.apply(lambda x:str_common_word(x['search_term'],x['product_title']), axis=1)
df_all['commons_in_desc'] = df_all.apply(lambda x:str_common_word(x['search_term'],x['product_description']), axis=1)

In [ ]:
# Levenshtein
import Levenshtein
df_all['dist_in_title'] = df_all.apply(lambda x:Levenshtein.ratio(x['search_term'],x['product_title']), axis=1)
df_all['dist_in_desc'] = df_all.apply(lambda x:Levenshtein.ratio(x['search_term'],x['product_description']), axis=1)

In [ ]:
# TF-IDF   
df_all['all_texts']=df_all['product_title'] + ' . ' + df_all['product_description']

In [ ]:
from gensim.utils import tokenize
from gensim.corpora.dictionary import Dictionary
dictionary = Dictionary(list(tokenize(x, errors='ignore')) for x in df_all['all_texts'].values)

In [ ]:
len(dictionary)

In [ ]:
class MyCorpus(object):
    def __iter__(self):
        for x in df_all['all_texts'].values:
            yield dictionary.doc2bow(list(tokenize(x, errors='ignore')))
corpus = MyCorpus()

In [ ]:
from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(corpus)
from gensim.similarities import MatrixSimilarity

# 先把刚刚那句话包装成一个方法
def to_tfidf(text):
    res = tfidf[dictionary.doc2bow(list(tokenize(text, errors='ignore')))]
    return res

# 然后，我们创造一个cosine similarity的比较方法
def cos_sim(text1, text2):
    tfidf1 = to_tfidf(text1)
    tfidf2 = to_tfidf(text2)
    index = MatrixSimilarity([tfidf1],num_features=len(dictionary))
    sim = index[tfidf2]
    # 本来sim输出是一个array，我们不需要一个array来表示，
    # 所以我们直接cast成一个float
    return float(sim[0])

In [ ]:
#TF-IDF similarity
df_all['tfidf_cos_sim_in_title'] = df_all.apply(lambda x: cos_sim(x['search_term'], x['product_title']), axis=1)
df_all['tfidf_cos_sim_in_desc'] = df_all.apply(lambda x: cos_sim(x['search_term'], x['product_description']), axis=1)

In [ ]:
#word2vec
import nltk
# nltk也是自带一个强大的句子分割器。
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
sentences = [tokenizer.tokenize(x) for x in df_all['all_texts'].values]
sentences = [y for x in sentences for y in x]

In [ ]:
from nltk.tokenize import word_tokenize
w2v_corpus = [word_tokenize(x) for x in sentences]

In [ ]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(w2v_corpus, size=128, window=5, min_count=5, workers=4)

In [ ]:
# 先拿到全部的vocabulary
vocab = model.vocab

# 得到任意text的vector
def get_vector(text):
    # 建立一个全是0的array
    res =np.zeros([128])
    count = 0
    for word in word_tokenize(text):
        if word in vocab:
            res += model[word]
            count += 1
    return res/count  

In [ ]:
from scipy import spatial
# 这里，我们再玩儿个新的方法，用scipy的spatial
def w2v_cos_sim(text1, text2):
    try:
        w2v1 = get_vector(text1)
        w2v2 = get_vector(text2)
        sim = 1 - spatial.distance.cosine(w2v1, w2v2)
        return float(sim)
    except:
        return float(0)
# 这里加个try exception，以防我们得到的vector是个[0,0,0,...]

In [ ]:
# word2vec similarity
df_all['w2v_cos_sim_in_title'] = df_all.apply(lambda x: w2v_cos_sim(x['search_term'], x['product_title']), axis=1)
df_all['w2v_cos_sim_in_desc'] = df_all.apply(lambda x: w2v_cos_sim(x['search_term'], x['product_description']), axis=1)

In [ ]:
#remove the text, leave number only
df = df_all.drop(['search_term','product_title','product_description','all_texts'], axis=1)

In [ ]:
df_train = df.iloc[:num_train]
df_test = df.iloc[num_train:]
id_test = df_test['id']

In [ ]:
y_train = df_train['relevance'].values
X_train = df_train.drop(['id','relevance'],axis=1).values
X_test = df_test.drop(['id','relevance'],axis=1).values

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score

params_rf = range(1,11)
test_score_rf = []
for param in params_rf:
    rfr = RandomForestRegressor(n_estimators=30, max_depth=param)
    test_score = np.sqrt(-cross_val_score(rfr, X_train, y_train, cv=5, scoring='mean_squared_error'))
    test_score_rf.append(np.mean(test_score))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(params_rf, test_score_rf)
plt.title("Random Forest-Number of trees");

In [ ]:
test_score_rf

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.cross_validation import cross_val_score

params_gbrt = range(1,11)
test_scores_gbrt = []
for param in params_gbrt:
    gbr = GradientBoostingRegressor(n_estimators=10, max_depth=param)
    test_score = np.sqrt(-cross_val_score(gbr, X_train, y_train, cv=5, scoring='mean_squared_error'))
    test_scores_gbrt.append(np.mean(test_score))

In [ ]:
plt.plot(params, test_scores)
plt.title("GBRT-number of stages");

In [ ]:
test_scores_gbrt

In [ ]:
rf = RandomForestRegressor(n_estimators=30, max_depth=9)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)